# Description

[describe]

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import utilities as util

# ignore depreciation warnings in this code
import warnings
warnings.filterwarnings("ignore")

# define the locations to save the figures into
fig_main_savepath='./paper_figures/main/'
fig_si_savepath='./paper_figures/si/'

# Load & Organize Data

### Berkeley (land/metric monthly data)

In [2]:
# define the berkeley daily maximum temperature data pathway
berkdpath='./data/berkeley/Berkeley_TMAX_land_188001_201712_daily_N96_365days_degC.nc'
# load in the daily maximum temperature rasters 
bddat=xr.open_dataset(berkdpath)

#### Organize time

In [3]:
# create the datetime grid for Berkeley monthly and daily data
timegridbd=util.get_dt64(bddat)

# add these back into the data structures
bddat['time']=timegridbd

### Get a Land-Sea Mask from the Berkeley data

In [4]:
# define the lsm raster pathway
lsmpath='./data/berkeley/berkeley_lsm.nc'

# load the land-sea mask into memory
lsm=xr.open_dataarray(lsmpath)

#### Replace 90S invalid data with missing

In [5]:
# set 90S equal to missing
bddat=util.set_N96_SouthPole_missing(bddat,'TMAX')

### GMST (global monthly timeseries, 3-year mean filtered via hadcrut_GMST.ipynb)

In [6]:
# load the data
gmstpath='./data/analysis/GMST-3yrmean.nc'
gmst = xr.open_dataset(gmstpath)
# get the timeseries from 1880-2017
gmstperiod=[1880,2017]
gmst_slice=util.dt64_yrslice(gmstperiod[0],gmstperiod[1])
GMST=gmst['GMST-3yrmean'].sel(time=gmst_slice)

### Find scenario Deltas from HadCRUT5 data

The scenarios for average temperature are:
* Counterfactual (1885-1915)
* Climate Base Period (1985-2015)
* IPCC Attributable Warming (2010-2019)

In [7]:
# calculate differences between periods
natural_shift=(gmst['Counterfactual_Delta']-gmst['CLIM_Delta']).values.flatten()[0]
forced107_shift=(gmst['Attributable_Delta']-gmst['CLIM_Delta']).values.flatten()[0]
# and print results to screen
print('Counterfactual GMST (1885-1915) anomaly from CLIM (1985-2015): '+str(np.round(natural_shift,2))+'C')
print('Attributable GMST (2010-2019) anomaly from CLIM (1985-2015): '+str(np.round(forced107_shift,2))+'C')
diff_ATT_minus_NAT=forced107_shift-natural_shift
print('Attributable minus Natural: +'+str(np.round(diff_ATT_minus_NAT,2))+'C')

Counterfactual GMST (1885-1915) anomaly from CLIM (1985-2015): -0.85C
Attributable GMST (2010-2019) anomaly from CLIM (1985-2015): 0.29C
Attributable minus Natural: +1.13C


### Calculate and Store Monthly medians

Now at every spatial location let's calculate the 50th percentile in each month from the daily data.

In [8]:
# define the median quantile we are calculating over
qi=0.5

In [9]:
# grab each month's data,
# e.g. month_idxs[1] IS JANUARY (idx=0:30,365:395,etc.), and so on for each month
month_idxs=bddat.groupby('time.month').groups
monthsi=np.arange(1,13)

In [ ]:
# loop over each of the months in the data set
for mi in monthsi:
    
    # find the median timeseries for a fixed month across the days in that month, for each year
    moni=bddat.isel(time=month_idxs[mi])
    store=moni.groupby('time.year').quantile(qi)
    del moni
    
    # create and assign time index, to assist merging, using central day/time of month as index
    tgrid=bddat['time'].isel(time=month_idxs[mi]).groupby('time.year').map(util.middle_element)
    store2=(
        store
            .assign_coords({'year': tgrid})
            .drop('time')
        )
    del store
    
    # merge into a single dataset
    if mi==1:
        q50i=store2['TMAX']
    else:
        q50i=xr.merge([q50i,store2])
    del store2
    # where are we in the loop?
    print('mon index='+str(mi))

# rename the time dimension
q50=(
    q50i
        .rename_dims({"year":"time"})      
        .rename({"year":"time"})   
    )
# q50['TMAX'].attrs
del q50i

mon index=1


In [ ]:
# q50['TMAX'].attrs

### Save out q50 array

In [ ]:
# save out to a netcdf file now that we have the Q50 data array
q50savepath='./data/berkeley/q50_monthly_1880-2017.nc'
q50.to_netcdf(q50savepath)

# Calculate Median Scale Factors ($\beta$): Regress q50 and GMST

We regress median maximum temperatures at every location (and for each month) against GMST, ignoring missing years in the regression. The resulting slope is the scale factor, $\beta$, used to scale the climatological distribution, to arrive at the forced and counterfactual distributions used in the observation-based attribution analysis.